**Содержание**<a id='toc0_'></a>    
- [Резюме модуля, темы](#toc1_)    
  - [В общих чертах: естественный язык и текст](#toc1_1_)    
    - [Правила бывают разные](#toc1_1_1_)    
    - [Общий алгоритм обработки текстов в программе](#toc1_1_2_)    
    - [Особенности обработки естественных языков](#toc1_1_3_)    
  - [В общих чертах: Лингвистический анализ](#toc1_2_)    
    - [Лингвистический анализ текстов.](#toc1_2_1_)    
  - [В общих чертах: Извлечение признаков](#toc1_3_)    
    - [Двоичный вектор](#toc1_3_1_)    
    - ["Взвешенные" векторные представления (TF-IDF и др.)](#toc1_3_2_)    
    - [N-граммы](#toc1_3_3_)    
    - [Методы понижения размерности](#toc1_3_4_)    
      - [Предиктивные модели текста (**языковые модели**)](#toc1_3_4_1_)    
  - [Выбор подходящей модели / представления](#toc1_4_)    
- [Прикладные задачи обработки текста и итоги](#toc2_)    
    - [Базовая задача — это тематическая классификация длинных](#toc2_1_1_)    
    - [Поисковые задачи обработки текста](#toc2_1_2_)    
    - [Извлечение структурированной информации](#toc2_1_3_)    
    - [Диалоговые системы](#toc2_1_4_)    
    - [Машинный перевод](#toc2_1_5_)    
    - [Эксплоративный анализ](#toc2_1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

Из проверочных заданий.

Напишите регулярное выражение, которое выделит в тексте только дробные числа (но не целые!).

In [2]:
import re

EVEN_NUMBERS_REGEX = re.compile(r'[0-9]+\.[0-9]+')

line = "1.23 1 1232.324 233.1 12..123 asd.123  "

all_even_numbers = EVEN_NUMBERS_REGEX.findall(line.strip())
print(' '.join(all_even_numbers))

1.23 1232.324 233.1


# <a id='toc1_'></a>[Резюме модуля, темы](#toc0_)

- Особенности разных языков и их влияние на методы анализ
    - Флексивность - изменчивость слов
    - Омномия - многозначность слов
    - Порядок слов - влияние на смысл
- Лингвистический анализ - структура текста на разных уровнях (от слов до дискурса)
- Извлечение признаков - подготовка текста к маш.обуч.
- Прикладные задачи - какие задачи решаются обработкой текстов

Классы рассмотренных алгоритмов:
- системы правил, регулярки, словари
- классическое маш.обуч. - линейные модели, ядерные методы, матричные факторизации
- Нейросети


## <a id='toc1_1_'></a>[В общих чертах: естественный язык и текст](#toc0_)

Естественный язык — в лингвистике и философии языка язык, используемый для общения людей (в отличие от формальных языков и других типов знаковых систем, также называемых языками в семиотике) и не созданный целенаправленно (в отличие от искусственных языков).

Формально, **язык** можно определить как множество цепочек символов из некоторого алфавита. Не всех цепочек, а только тех, которые удовлетворяют
некоторым правилам. 

Отдельно взятая цепочка, построенная по этим правилам — это и есть **текст**. 

**Алфавит** — это множество символов, которые можно использовать для построения текстов. 

По идее, каждая цепочка должна нести какую-то информацию, но на практике — нет.

*"Глокая куздра штеко бодланула бокра и кудрячит бокрёнка"* (Л.В.Шерба, 1930-е) 

- написано по правилам русского языка, смысл ни одного слова не известен, но общий смысл у текста есть, он несет информацию.

### <a id='toc1_1_1_'></a>[Правила бывают разные](#toc0_)

Обычно их разбивают на уровни — от низкоуровневых до высокоуровневых. 

- Графематические правила — это правила разбиения текста на отдельные единицы: на предложения и слова. 
- Морфологические правила управляют процессом словообразования и словоизменения. 
- Синтаксические правила управляют согласованием слов, чтобы было понятно, что к чему относится. 
- Семантические правила управляют процессом передачи смысла посредством правильного использования всех вышеперечисленных правил.
- Стилистические правила говорят нам, что уместно в данной конкретной ситуации, что неуместно и как мы должны говорить, если хотим вызвать какие-то
определённые эмоции у слушателя.

Язык — это неотделимая часть культуры, в которой он используется и развивается. И, вроде бы, один и тот же язык может иметь различия на разных уровнях правил
— например, "испанский" испанский и "южно-американский" испанский имеют общий синтаксис, но стилистика употребления слов них разная.

### <a id='toc1_1_2_'></a>[Общий алгоритм обработки текстов в программе](#toc0_)

Как правило, процесс строится снизу вверх, от графематики к семантике, и дальше. И, как правило, на каждом шаге используются результаты всех предыдущих шагов. 

1. Сырой текст, цепочка символов

    `"Мама мыла раму. Потом мы пошли гулять."`

2. На выходе графематического анализа мы уже получаем "список списков", каждый список
представляет предложение, а внутри предложения есть отдельные "токены" или "лексемы".

    `[["Мама", "мыла", "раму"], ["Потом", "мы", "пошли", "гулять"]]`

3. Следующий уровень — это морфологический анализ. Он обогащает каждую лексему информацией о словоформе, то есть о морфологических признаках и начальной форме слова.

    `[[Token("Мама", init="мама", pos="сущ"), Token("мыла", init="мыть", pos="глаг"), ...], ...]`

4. На выходе синтаксического анализа мы получаем дерево, характеризующие подчинение слов. В структуре данных оно может задаваться, например, ссылкой на родительский узел. В нашем
примере, синтаксическое дерево может иметь следующую конструкцию: главное слово в предложении — глагол ("мыла"). Глаголу подчиняются главные слова в словосочетаниях — как правило,
это существительные ("мама", "раму"). А существительным подчиняются определения ("оконную"). Таким образом, мы получили дерево для нашего примера, и в структуре данных оно задаётся ссылкой на номер
токена в предложении, например.

    `[[Token("Мама", init="мама", pos="сущ", ), Token("мыла", init="мыть", pos="глаг", synt_parent=None), ...], ...]`

5. Семантический анализ строит ещё более многосвязную структуру — граф или семантическую сеть. Этот граф описывает ситуацию на самом верхнем уровне: что происходит, какие есть участники и какие роли они выполняют. Вместо глагола может использоваться также отглагольное существительное, также такие слова называют предикатными словами. Кроме предикатного слова в предложении есть какие-то аргументы, "мама", "оконная рама" и "тряпка".  На самом деле, рёбра обычно проводятся между главным словом и аргументами, а также между аргументами напрямую — это называется "семантические отношения".

6. Результаты всех выполненных шагов агрегируются для того, чтобы решать конечную задачу — то, зачем мы взялись за обработку текста.

### <a id='toc1_1_3_'></a>[Особенности обработки естественных языков](#toc0_)


Первая особенность заключается в том, что эти языки никем специально не составлялись, а появились и эволюционировали в результате потребности людей в коммуникации. Никто полностью не
знает правил естественных языков существует множество неписаных правил, которые меняются от региона к региону и со временем, особенно сейчас, в эпоху глобализации. Не обязательно соблюдать правила. Если вы скажете какую-то фразу неправильно, то, скорее всего, вас всё равно поймут (до определённого предела). А ещё, правила в естественном языке неоднозначны — позже мы рассмотрим парочку таких примеров. Естественным языкам часто противопоставляются формальные языки. Например, языки программирования или язык математических формул. Такие языки подчиняются строгим правилам, которые задаются,
например, порождающими формальными грамматиками. Для таких языков процесс разбора и поиска ошибок детерминирован и есть эффективные алгоритмы, реализующие этот процесс.

**Лингвистическая типология**

Языки отличаются:
- в правилах образования и изменения слов
- в правилах связывания слов друг с другом
- в способах передачи смысла

На практике примеры отличий англ./рус.:
- флексивность (словоизменчивость) - слабая/сильная
- смысловая омономия - высокая/высокая
- частеречная омономия - сильная/умеренная
- порядок слов - фиксированный/свободный

Стемминг - это процесс нахождения основы слова для заданного исходного слова. Основа слова не обязательно совпадает с морфологическим корнем слова. Задача нахождения основы слова представляет собой давнюю проблему в области компьютерных наук. **Основа - не обязательно вообще осысленное слово**

Лемматизация - процесс приведения словоформы к лемме — её нормальной (словарной) форме. **Нормальная форма - всегда осысленное слово**


**Смысловая неоднозначность словоупотреблений**

Одно и то же слово, употреблённое в разных контекстах, может иметь разный смысл. Такое явление называется **омонимей**. 

Характерно для большинства языков.

Снять смысловую неоднозначность призваны соответствующие методы. На английском эта задача называется "word sense disambiguation".

**Частеречная омонимия**

Это как смысловая, только неоднозначность заключается не в выборе смысла, а в выборе части речи. Немножко мы уже коснулись этой темы. Например, в предложении "мама мыла раму", какую часть речи имеет слово "мыла"? А в предложении "в ванной не было мыла"?

На английском это звучит как "part of speech tagging", или "POS-tagging"

**Вариативность порядка слов**

В русском языке наиболее выражена, в англ. - наименее.


## <a id='toc1_2_'></a>[В общих чертах: Лингвистический анализ](#toc0_)

Можно выделить три высокоуровневые группы задач обработки текстов на естественном языке. 

1. **Первая группа** — это лингвистический анализ. Методы из области лингвистического анализа направлены на разбор структуры текста на разных уровнях. 

2. **Вторая группа** — методы извлечения признаков из текстов. Они частично пересекаются с первой группой, но, в то время как задачи лингвистического анализа — это самостоятельные задачи, задачи извлечения признаков всегда предшествуют применению методов машинного обучения. 

3. **Третья группа** — прикладные задачи, они ближе к бизнесу, к пользователю. Как правило, для их решения используются методы из первых групп плюс какие-то специальные надстройки. 


### <a id='toc1_2_1_'></a>[Лингвистический анализ текстов.](#toc0_)

Цель: извлечение структуры текста. 

Как мы уже говорили ранее, решаются задачи в порядке от низкоуровневых к более высокоуровневым. 

1. **Подготовка**

- графематический анализ (разбиваем текст на предложение и токены)
 
        Графематический анализ принимает на вход сырой текст и возвращает разбиение на токены и предложения. Для его реализации, в простейшем случае, используются регулярные выражения. Мы просто находим все разделительные символы — пробелы и знаки препинания, а потом из них отбираем только те, которые соответствуют окончанию предложения. Часто не очень просто определить, какая точка обозначает окончание предложения, а какая — сокращение. Например, в этом предложении есть фамилия, имя, отчество. Имя и отчество сокращены, а дальше следует "точка". Какая из этих трёх точек обозначает конец предложения? Чтобы убрать эту неоднозначность на практике, после регулярных выражений, используются вероятностные модели, описывающие совместное распределение вероятностей меток токенов. К таким моделям относятся случайные условные поля (Conditional Random Fields) или скрытые марковские модели (далее не рассматриваются, для самостоятельного изучения).

2. **Анализ отдельных предложений** (от морфологии до семантики)

- морфология

        Морфологический анализ работает с отдельными токенами. Для каждого токена анализатор предлагает набор сочетаний характеристик, к которым относится часть речи, падеж, число, начальная форма... При этом анализатор не может выбрать только одно сочетание характеристик, потому что для этого нужно привлекать более широкий контекст.

- POS-теггинг (разбор частеречной омонимии)

        Частиречная омонимия морфологическим анализов не разрешается. Например, для слова "мыла" вот в такой форме анализатор предложит два варианта: это прошедшее время и женский род глагола "мыть", а также родительный падеж и единственное число "мыло". 
        Для реализации используются разные методы, самый простой из которых — словарный. Для частотных словоформ все возможные варианты уже известны наперёд, надо их только достать. А для неизвестных или редких слов используются регулярные выражения, системы правил, которые по окончанию слова пытаются предположить, как должна выглядеть начальная форма этого слова, а также морфологические характеристики этой словоформы. 
        Логичный следующий шаг — это для каждого токена, из всех вариантов, предложенных для него морфологическим анализатором, выбрать только один, то есть разрешить частиречную омонимию. Омонимия снимается в рамках одного предложения. Такого контекста на практике достаточно почти всегда. В результате из всего множества вариантов выбирается только один — например, "глагол". Для решения этой задачи существуют методы, основанные на правилах, в том числе полученных с помощью машинного обучения. Однако более современный и более распространённый вариант — это вероятностные модели и последовательности, то есть условные случайные поля и марковские модели. Когда размеченный корпус достаточно большой, рекуррентные нейросети позволяют улучшить качество ещё сильнее.

- извлечение именованных сущностей

        В рамках задачи извлечения именованных сущностей, необходимо для каждого словосочетания или короткой последовательности токенов, установить класс. Например, "организация" ("Рога и копыта", "Хвосты и лапы" и т.п.). Другой класс именованной сущности — это имя человека ("Василий Петров" и т.д.). Могут быть географические наименования и ещё множество разных вариантов. 
        
        Самый простой метод — словарный. Он позволяет достаточно надёжно выделить широко известные и при этом уникальные имена, такие как "Samsung" или "Сбербанк". Эти слова не имеют других смыслов или функции, кроме обозначения данных конкретных организаций. 
        
        Для некоторых типов сущностей (например, адрес электронной почты или номер телефона) хорошо подходят регулярные выражения. Однако на практике для повышения точности, то есть уменьшения количества ложноположительных предсказаний, необходимо применять вероятностные модели. Сначала по словарю или с помощью регулярных выражений выделяем какой-то набор кандидатов, а затем мы применяем вероятностную модель, чтобы выбрать наиболее подходящее сочетание классов для каждого токена. 
        
        Нейросети позволяют объединить все эти подходы в один алгоритм, все шаги которого настраиваются совместно с помощью методов оптимизации. 
        
- синтаксический анализ

        Теперь мы можем выполнить синтаксический анализ, результатом которого является, например, дерево зависимостей. В дереве зависимостей для каждого токена указана ссылка на его предка. 
        
        Самый распространённый подход для синтаксического анализа сейчас — это хорошо известные классические алгоритмы класса "сдвиг-свёртка" (shift-reduce анализаторы), которые используются для синтаксического анализа не только естественных языков, но и языков программирования. Однако существенное отличие в их вариантах для естественных языков заключается в том, что для принятия решений внутри анализатора используются обучаемые классификаторы, которые могут быть как линейными моделями, так и нейросетями. Именно так работают одни из лучших известных сейчас реализаций синтаксических анализаторов — MaltParser или SyntaxNet.

- семантический анализ

        Следующий шаг — семантический анализ. Мы работаем только с простыми предложениями. Сложные предложения сначала разбиваются на простые. 
        
        Рассмотрим пример - Гендиретора ООО "Рога и копыта" Василий Петров из Москвы подписал соглашение об объединении с петербургской ООО "Хвосты и лапы". 
        
        1. Первый шаг — это выделение главного слова, предиката. Как правило, это глагол или отглагольное существительное. В данном примере у нас два предиката. Первый — это "подписал" а второй — это "объединение". 
        
        2. Когда предикат найден, мы можем понять структуру ситуации, то есть выделить набор семантических ролей, которые мы ожидаем рядом с этим предикатом увидеть. Существуют две семантические роли, которые есть практически у всех глаголов — это субъект (тот, кто делает) и объект (к кому действия применяется). В нашем случае аргумента три: первый — это "гендиректор ООО "Рога и копыта", Василий Петров из Москвы" (такой большой один аргумент); второй аргумент — это "соглашение", третий аргумент — это "петербургское ООО "Хвосты и лапы". Напомню, что у нас здесь есть два простых предложения, вернее 2 клаусы/клаузы/clause (элементарное предложение, предикация) - в синтаксисе: составляющая, вершиной которой является глагол либо, в случае отсутствия глагола, связка или элемент, играющий её роль. 
        
        3. На следующем шаге каждому аргументу назначается роль — например, "гендиректор" — это субъект, "соглашение" — это объект. Часто в качестве вспомогательного инструмента применяются словари, особенно для глаголов, потому что глаголов не очень много, и мы сразу можем понять по глаголу, какая ролевая структура ему должна соответствовать. Аргументы, как правило, выделяются с помощью правил, применяемых к синтаксическому дереву. Более современные подходы основаны на методах из области синтаксического анализа. Они объединяют все эти шаги воедино. 
        
- извлечение отношений между сущностями внутри предложения

        Следующий шаг — извлечь отношения между сущностями. К таким отношениям можно отнести, например, часть/целое или принадлежит, или предшествует/следует, и так далее. В нашем примере можно выделить отношения между, например, сущностями "Василий Петров" и "Рога и копыта". Отношение описывается словом "гендиректор". Таким образом мы знаем, что Василий Петров — это гендиректор ООО "Рога и копыта". Ещё мы можем выделить другое отношение — это то, что "Рога и копыта" находится в Москве. Общий подход к решению такого класса задач — выделить всевозможные сущности в предложении, а затем применять некоторый классификатор ко всем возможным их парам.

3. **Анализ целых текстов**

- разрешение анафорических связей

    Анафорические связи — это когда местоимение ссылается на какой-то объект, ранее упоминавшейся, или тот, который будет упоминаться в дальнейшем. В результате применения метода разрешения анафорических связей мы ожидаем, что будут установлены связи между такими местоимениями и словами, на которые они ссылаются — референтами. 
    
    Рассмотрим пример из двух предложений. 
    
        "Марк надел пальто и вышел из дома осенним вечером. Он не очень любил холод осени, но ее красота согревала."

    Здесь есть два местоимения и есть несколько вариантов референтов, например, для слова "он", можно выделить все слова мужского рода: "вечер", "дом" или "Марк". Соответственно задача заключается в том, чтобы из всех этих вариантов референтов выбрать только один. В данном случае, явно, он — это "Марк". 
    
    Во втором случае всё попроще, потому что у нас только одно слово женского рода в предыдущем предложении: "осень". Подход в целом такой же как и в предыдущей задаче — задаче извлечения связей между сущностями. 
    
    Мы сначала извлекаем все возможные пары "местоимение и референт", а потом применяем классификатор, который нам говорит, кто на кого ссылается, действительно.

- дискурсивный анализ

    Дискурсивный анализ — это как синтаксический анализ, только для документа в целом. Идея в том, чтобы получить структуру повествования или аргументации — что за чем следует, что является подтверждением, что — опровержением, что с чем сравнивается, и так далее. 
    
    Дискурс начинает работать, когда у нас есть несколько простых предложений. Например, у нас есть следующее предложение — 
    
        По сравнению с X, Y лучше подходит для Z, потому что A и B.
    
    в нём есть следующие части — 'по сравнению с "x", "y" лучше подходит для "z", потому что "а" и "b"'. 
    
    Между этими частями мы можем установить, например, следующие связи: "сравнение", "аргументация", а также "однородность". 
    
    Так как задача выглядит как синтаксический разбор, логично её решать такими же методами. Действительно, так на практике и делают. На практике — это анализаторы класса "сдвиг-свёртка", в которых внутри работают классификаторы, основанные на машинном обучении.

4. **Генерация текста** (побочная задача, которая может включать все или часть предыдущих задач)

Давайте ещё поверхностно коснёмся задачи генерации текста. Она достаточно сложная и, вообще, она относится скорее к прикладным задачм, чем к лингвистическому анализу. 

На вход генератору, как правило, подаётся структурированное описание того, что нужно сказать — что нужно преобразовать в текст на естественном языке.
 
- Один из способов описать ситуацию — это семантический фрейм. Это объект, у которого указано, какое действие кто делает, с чем делают, и так далее. 
- Другой способ задать смысл предложения — это вектор признаков, полученный из нейросети или какими-то другими способами. 

Ожидаемо, на выходе получается текст в виде последовательности лексем или прямо в виде строки. 

- Исторически, первый и один из самых практичных подходов — это генерация через поиск. Мы не создаём новые тексты, мы просто стараемся найти достаточно подходящий текст из уже имеющихся в базе. Такой подход часто используется в чат-ботах. 

- Альтернативный подход — это генерация по шаблону. Шаблоны задаются программистами. Этот метод позволяет упоминать в тексте сущности, о которых речь шла ранее в диалоге (например, если речь идёт о чат-ботах). 

- Самый современный, но пока что плохо изученный — это генерация текста с помощью нейросетей. Итак, мы начали рассматривать задачи обработки текстов на естественном языке, а именно — задачи лингвистического анализа.

Пример:

Выберите все задачи лингвистического анализа, которые можно решать в два этапа:

    выделение всех кандидатов (это могут быть слова, токены, сущности состоящие из нескольких слов)
    обучение классификатора, принимающего пару кандидатов, и предсказывающего класс отношения между ними.

Ответ:

Т.к. речь об отношениях, то подходят только методы, которые работают со структурами типа графа/дерева

Подходят:

- Извлечение отношений между сущностями внутри предложения (построение графа, описывающего отношения между упоминаемыми в предложении объектами)
- Синтаксический анализ (построение дерева, описывающего структуру фраз, грамматическое подчинение и согласование слов)
    *В синтаксическом анализе такой подход может использоваться, но на практике не вполне эффективен, потому что классификатор, принимающий на вход пару кандидатов, не может учитывать остальные кандидаты и отношения между ними.*
- Семантический анализ (построение графа, описывающего высокоуровневую ситуацию - кто какое действие с чем выполняет, когда, почему и т.п.; в вершинах такого графа обычно находятся отдельные слова или фразы)
- Разрешение анафорических связей (англ. anaphora resolution: построение графа, в котором рёбра связывают референта и ссылающиеся на него местоимения или словосочетания; референт - некоторый объект, который может именоваться в тексте разными способами)

Не подходят:
- Морфологический анализ (предсказание морфологических характеристик отдельных токенов)
- Извлечение именованных сущностей (англ. Named Entity Recognition: классификация токенов, классы соответствуют видам сущностей)
- POS-теггинг (снятие частеречной неоднозначности, т.е. выбор наиболее правдоподобного сочетания частей речи для токенов в предложении)

## <a id='toc1_3_'></a>[В общих чертах: Извлечение признаков](#toc0_)

Центральное положение в решении задачи извлечения признаков занимают методы, основанные на машинном обучении (так уж сейчас сложилось). 

Такие методы (классификаторы) получают на вход специальные структуры данных — 
    
- чаще всего это вектора или матрицы (тензоры)
- иногда алгоритмы принимают на вход графы или деревья. 

**Зачем рассматривать вообще задачу извлечения признаков отдельно от конечной задачи?**

На самом деле на это есть множество причин. 

1. Значительная часть методов извлечения признаков не требует разметки. 

        Следовательно, мы можем применить методы к большим неразмеченным корпусам, получить хорошие признаки, а потом уже работать с небольшой размеченной выборкой. 

2. Имеется возможность извлечь признаки один раз, а потом поверх них накручивать классификаторы для решения разных задач. 

3. Бывает, что можно получить неплохое представление (текстов), не используя машинного обучения вообще. 

        В первую очередь, рассмотрим разреженное векторное представление, популярное в классических подходах, а затем перейдём к более сложным нейросетевым и ядерным подходам. 

Далее дается обзор и терминология по теме для дальнейшего самостоятельного изучения.

### <a id='toc1_3_1_'></a>[Двоичный вектор](#toc0_)
Это очень простой метод. Он подходит, в том числе, тогда, когда тексты сильно отличаются по длине.

- Элементы вектора соответствуют отдельным словам. Элемент равен "1", если слово в документе присутствует, и "0", если нет. 
- Размерность векторного пространства, получаемого таким образом, достаточно большая (равна размеру словаря), и поэтому почти любые классы линейно разделимы, и линейные модели хорошо работают на таких пространствах (можно сказать, что тут "проклятие размерности" работает в плюс). 

*Главная проблема* заключается в том, что и частотные слова общеупотребимые, и специальная лексика, имеют одинаковый вес, то есть мы не знаем, как много используется каждое слово в документе. По этой же причине метод чувствителен к опечаткам, случайным словам, и так далее. 

*Общая проблема* разреженных векторных моделей — в так называемом "предположении о независимости". 

Элементы вектора, соответствующие разным словам заполняются независимо, и таким образом мы теряем информацию о том, что какие-то два слова являются синонимами и если одно встретилось, то, возможно, может встретиться и другое. 

Из-за этого модель может не очень хорошо обобщаться на новые данные. И это происходит не из-за переобучения, а из-за того, что просто признаки не очень хорошие, в них просто нет такой информации. 

В то время как высокая размерность вектора может рассматриваться как преимущество, она может также являться и недостатком, потому что если обучающая выборка не очень большая, а признаков при таком подходе получается очень много — сотни тысяч, мы можем переобучиться даже если работаем с простой линейной моделью.

    В двоичном векторном представлении противоположные по смыслу тексты могут выглядеть абсолютно одинаково (Казнить нельзя, помиловать == Казнить, нельзя помиловать), а схожие по смыслу - абсолютно по разному (Нельзя не помиловать != Обязательно освободить (ни по одной координате вектора не совпадет)). Тем не менее, для тематической классификации это представление работает весьма неплохо.

[1] Christopher D. Manning, Prabhakar Raghavan and Hinrich Schütze, Introduction to Information Retrieval, Cambridge University Press. 2008. https://nlp.stanford.edu/IR-book/

### <a id='toc1_3_2_'></a>["Взвешенные" векторные представления (TF-IDF и др.)](#toc0_)

Логичный следующий шаг — это исправить самый главный недостаток бинарного вектора — кодировать не нулями и единичками, а как-то взвесить. 

Самый простой источник для взвешивания — это частотность слова, причём для каждого слова мы можем посчитать не только частоту его использования в одном документе, но и частоту его использования в корпусе в целом, а потом их как-то скомбинировать (например, с помощью формулы TF-IDF). 

Преимущества: 
- Итак, это по-прежнему достаточно простая модель. 
- Если перед вами стоит задача тематической классификации длинных текстов, то этот подход очень хорошо будет работать и вполне возможно, что вы не сможете улучшить качество классификации, используя сложные методы (например, нейросети). 
- По-прежнему мы сохраняем основные преимущества предыдущего подхода, то есть линейную разделимость данных
- Также мы можем играться с весами, искусственно завысив веса для каких-то слов, или занизив — таким образом, закладывая в модель наши знания о том, что должно быть важно для принятия решения, а что — нет. 

Недостатки:
- TF-IDF чувствителен к опечаткам (высокая чувствительность к шуму) в обучающей выборке и другим редким словам, которые, тем не менее, достаточно частотные, чтобы не быть отфильтрованными. 
- Предположение о независимости словоупотреблений (кружка/чашка никак не связаны => нет (почти нет) обобщающей способности у метода)
- Слишком большая размерность вектора (нужна большая выборка, чтоб не было переобучения).

### <a id='toc1_3_3_'></a>[N-граммы](#toc0_)

Иногда выгоднее считать не слова, а **N-граммы - последовательности подряд идущих символов или токенов**. 

N-граммы бывают:
- символьными

        3-граммы: днё, нём, ём_, м_м, _ма, мам

- пословными 

        2-граммы: день мама, мама мыть, мыть рама


N-граммы — это достаточно популярный трюк, который используется много где, 
- в том числе с TF-IDF, 
- а также в современных методах построения плотных векторных представлений и дистрибутивной семантики — например, в модели **FastText**. 

Преимущества:

- простота. Использование N-грамм не сильно усложняет модель. 
- относительная устойчивость к опечаткам, а также словоизменению, то есть мы можем до какой-то степени обойтись без исправления опечаток, а также без сложных алгоритмов нормализации текста — например, лемматизации и морфологического анализа. 
- пословные N-граммы — более специфичны по сравнению с отдельными словами, то есть они встречаются реже, но при этом (если они встречаются) являются более сильным фактором. И поэтому они могут лучше описывать особенности тематики текстов. 

Недостатки:

Как всегда, недостатки подхода следуют из его преимуществ. 

- Размерность пространства растёт очень быстро, а вектора получаются очень разреженными (чем больше N, тем реже соответствующая N-грамма встречается)
- Остаются и другие недостатки, присущие предыдущим векторным представлениям (предположение о независимости разных N-грамм). 

### <a id='toc1_3_4_'></a>[Методы понижения размерности](#toc0_)

**Хорошая новость том, что вектора можно сжать**, что может дать МНОГОкратную экономию памяти для представлений текстов в памяти

Например, если у нас есть гигантская матрица, описывающая встречаемость слов в документах (размерность Docs * Words), то мы её можем факторизовать, то есть представить в виде произведения двух матриц меньшей размерности, меньшего ранга. Например, первая матрица будет представлять документы в некотором латентном (?) пространстве (кодирующем тематики текстов), а вторая матрица будет представлять слова в том же самом пространстве. 

Т.е. надо найти такое разложение разреженной матрицы (в терминах размерностей):

        Docs * Words = (Docs * Topics) x (Topics * Words)

\*) Есть переменные наблюдаемые, а есть скрытые (их ещё называют латентными). Наблюдаемые переменные - это те переменные, для вычисления которых не надо строить какую-то особую модель. В случае с текстами к таким переменным относятся, например, частоты слов, которые в тексте встречаются. К латентным переменным можно отнести, например, тематики (когда текст на 50% про котиков, на 30% про психологию и ещё на 20% ещё про что-то). 
Другими словами, латентные пространства (или пространство латентных переменных) - это когда мы пытаемся описать объекты (тексты) с помощью каких-то переменных (признаков), которые не вычисляются непосредственно из текста, а требуют построения специальной модели (тематической модели, нейросети и т.п.).


Итого, методы сжатия векторых представлений тектов:

- Матричные разложения и тематическое моделирование: 

        - SVD (опорные вектора), 
        - PLSA/PLSI (Вероятностный скрытый семантический анализ (PLSA), вероятностное скрытое семантическое индексирование (PLSI)), 
        - LDA (Латентное размещение Дирихле), 
        - ARTM (Аддитивная регуляризация тематических моделей).

- Предиктивные модели дистрибутивной семантики[1] или дистрибутивно-семантические модели которые обучаются предсказывать соседние слова (в текстах) для данного слова (Word2Vec, FastText)

- Предиктивные модели текста или **языковые модели** (BERT, ELMo, OpenAI Transformer)

[1] Дистрибутивно-семантические модели для русского языка: https://rusvectores.org/ru/

\*\*) После токенизации всех документов в корпусе строится словарь, содержащий для каждого уникального токена количество его употреблений в корпусе. Затем из этого словаря удаляются самые редкие слова.

Это может быть нужно:


- Чтобы уменьшить риск переобучения

- Чтобы сэкономить память, требуемую для размещения датасета и модели

- Чтобы убрать слова, содержащие опечатки

#### <a id='toc1_3_4_1_'></a>[Предиктивные модели текста (**языковые модели**)](#toc0_)

Наиболее сложный и мощный класс.

Они предсказывают следующее слово по известному префиксу предложения, и, как правило, реализуются глубокими нейронными сетями. 

Хорошая новость в том, что, несмотря на то, что методы — обучаемые, разметка им не требуется. 

Такие методы называются методами с самонаблюдением (self-supervised), то есть целевая переменная берётся из самих анализируемых объектов — например, это просто следующее слово. 

То есть, руками размечать ничего не надо, и мы можем автоматически обработать гигантские объёмы текста. 

Методы из этой группы, особенно — основанные на языковых моделях, позволяют достичь наилучшего качества на некоторых сложных задачах. Наконец-то мы отвергли предположение о независимости словоупотреблений и наконец-то можем начать работать с синонимами. 

Так как в основе этих методов лежит оптимизация, мы можем настраивать веса моделей с учётом сразу нескольких критериев, таким образом делая признаки полезными сразу для нескольких задач. Это очень популярное сейчас направление — оно называется "multitask learning". Но за все надо платить — эти модели гораздо дороже с точки зрения объёмов вычислений, особенно нейросетевые модели. И другая сложность заключается в том, что им нужны гигантские объёмы текста. Еcли у вас меньше нескольких миллионов документов, обучить с нуля качественную языковую модель у вас, скорее всего, не получится, однако сейчас в интернете доступны веса моделей, уже обученных на некоторых больших корпусах, их можно просто скачать и использовать в своём проекте (это здорово).

Итого, **преимущества**:

- размеченная выборка не требуется
- обеспечивают наилучшее качество на некоторых классах задач
- учитывают совместную встречаемость слов и их отношений в тексте
- использование одиних и тех же готовых/обученных моделей для решения разных задач

**Недостатки**:

- вычислительная сложность
- необходимость многомиллионных обучающих выборок

## <a id='toc1_4_'></a>[Выбор подходящей модели / представления](#toc0_)

Надо помнить, что правило 80/20 никто не отменял:[1] если вам надо быстро получить хоть какое-то решение, возможно, нет смысла заморачиваться с языковыми моделями или нейросетями вообще и лучше начать с векторной модели и линейных классификаторов, в части случаев этого будет вполне достаточно. 

Другой недостаток в том, что процесс обучения не всегда стабилен. Может потребоваться перебор гиперпараметров или несколько запусков. 

Когда мы работаем с векторами, у нас есть миллион способов оценить их сходство. Один из таких способов — это скалярное произведение. Физический смысл скалярного произведения, как вы все, конечно, знаете — это косинус угла между двумя векторами. Однако, когда мы работаем с текстами, в этом есть небольшая проблема. Текст — это сложная нелинейная структура, и если мы её представляем вектором, мы неизбежно **теряем часть информации**. 

Поэтому вполне естественная идея — представить текст как **граф или семантическую сеть**. Отлично, теперь дело за малым — нужно определить хорошую операцию сходства на графах. Такую операцию принято называть **"ядром"**. 

- Например, мы можем запускать некоторое количество случайных обходов в этих графах, а потом смотреть, насколько обходы, полученные по разным графам, пересекаются друг с другом: чем больше пересекаются, тем больше графы похожи друг на друга. 

- Или ядро можно выучить (с помощью нейросети, например, tree recursive neural network). 

Когда мы нашли подходящее ядро, мы можем использовать относительно простые модели — 

- SVM - метод опорных векторов[2]
- KNN - поиск ближайших соседей[3]. 


Преимущества:

Хорошая новость в том, что, даже не имея разметки, мы можем получить достаточно выразительный способ представления текстов. Для некоторых сложных задач — например тех, в которых размечать большие корпуса достаточно дорого, ядерные методы позволяют получить наилучшее на сегодняшний день качество. К таким областям, например, относится извлечение информации из медицинских текстов. Посредством ядра мы можем заложить в модель наши экспертные знания о предметной области, тем самым ещё сильнее сокращая требования к обучающей выборке. К сожалению, ядерные методы, с вычислительной точки зрения, иногда даже медленнее нейросетевых, так как они хуже распараллеливаются с помощью графических ускорителей. 

Ну и, вполне логично, модель будет ровно настолько хорошей, насколько хорошее ядро вы придумали.[4]

[1] Закон Парето или правило [80/20](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%9F%D0%B0%D1%80%D0%B5%D1%82%D0%BE)

[2] SVM с нелинейным ядром

[3] Задача поиска ближайшего соседа

[4] [Kriege, Nils M., Fredrik D. Johansson, and Christopher Morris. "A Survey on Graph Kernels." arXiv preprint arXiv:1903.11835 (2019)](https://arxiv.org/abs/1903.11835)

# <a id='toc2_'></a>[Прикладные задачи обработки текста и итоги](#toc0_)


К прикладным задачам мы отнесли те, которые можно так или иначе положить в основу продукта. 

1. Классификация [2] 
    - тематическая классификация длинных тектов
    - классификация коротких текстов (по тональности, интенции и т.п.)

2. Поиск 
    - по разным видам запросов
    - текста по изображению, изображения по тексту
    - похожих тектов
    - вопросно-ответный поиск

3. Извлечение структурированной информации[1] 
    - важная задача, связанная с наполнением баз данных, автоматизацией, переносом компетенций, процессом принятия решений... 
    
4. Диалоговые системы (помощники, чат-боты и т.п.)

5. Машинный перевод

6. Эксплоративный анализ 
    - задача без чётких критериев качества, когда надо понять, что вообще происходит. 
    


### <a id='toc2_1_1_'></a>[Базовая задача — это тематическая классификация длинных](#toc0_)

Подходы для длинных и коротких тектов принципиально отличаются.

**Дано:**

- коллекция текстов, для которых проставлены метки. 
- длина текстов должна позволять набрать хоть какую-то статистику. 
- природа меток такова — они зависят от состава текста в целом, а не от отдельных фраз или формулировок. 

**Алгоритмы:**

- Самый подходящий алгоритм для этой задачи — это линейный классификатор с разреженными признаками, взвешенными по частоте[3] (например, TF-IDF). 

- Нейросети тоже можно применять, но они часто не дают большого прироста в качестве. 

**Дано:**
- коллекция коротких текстов: частоты слов уже не подсчитать,
- метки чаще связаны со структурой фраз, а не с тематикой. 

Например, по составу слов в предложении (Готовился к плохому, а получилось хорошо) достаточно сложно определить его эмоциональную окраску, однако если мы рассмотрим связи тональных слов с теми словами, которые они определяют, станет понятно, что предложение, в целом, положительное. 

**Алгоритмы:**

- нейросети работают уже существенно лучше чем с длинными текстам — особенно, когда есть большой размеченный корпус. 
- если меток не очень много, ядерные методы могут сработать лучше. 
- если же разметки нет вообще, то, возможно, проще запрограммировать правила, анализирующие структуру предложений[4], обходящие синтаксические деревья или семантические сети[5] и принимающее решения.

[1] Christopher D. Manning, Prabhakar Raghavan and Hinrich Schütze, Introduction to Information Retrieval, Cambridge University Press. 2008. https://nlp.stanford.edu/IR-book/  
[2] Список статей с кодом по тематике классификации текстов https://paperswithcode.com/task/text-classification  
[3] https://en.wikipedia.org/wiki/Bag-of-words_model  
[4] Лексико-синтаксические шаблоны в системе GATE https://gate.ac.uk/sale/tao/splitch8.html#chap:jape  
[5] Лексико-синтаксические шаблоны Томита-парсера https://github.com/yandex/tomita-parser/blob/master/docs/ru/tutorial/basic-rules.md

### <a id='toc2_1_2_'></a>[Поисковые задачи обработки текста](#toc0_)

Самый популярный вид поиска — по запросу. При этом, запрос не обязательно является связным текстом на каком-то языке, чаще это просто набор ключевых слов. 

**Дано:**
- коллекция текстов
- статистика посещения пользователей (поиска), по которой мы можем обучить оценку релевантности (желательно, но не обязательно). 

**Алгоритмы:**

Базовый подход — это векторная модель текста вместе с классическими формулами оценки релевантности — как правило, это всё основано на частотах слов (TF-IDF). BM25

Следующее логичное развитие — это расширить частотную модель с помощью дистрибутивной семантики, то есть "эмбеддингами" слов (наподобие word2vec). 

Использовать глубокий лингвистический анализ и графовые ядра, чтобы точнее сопоставлять тексты. 

Когда накапливается статистика посещения пользователей, появляется возможность применять обучаемые методы ранжирования (на английском эта задача назывался "learning to rank"). Самые популярные алгоритмы для ранжирования — это градиентный бустинг и нейросети. 


Более сложная постановка задачи поиска — поиск между модальностями, например, когда мы ищем картинку по тексту, или наоборот[1]. 

**Дано:**
- это коллекция мультимедийных документов, состоящая из текста и иллюстраций, например — веб-страницы. 

**Алгоритмы:**

**Исторически** первый подход работал через текстовый поиск, ключевой момент здесь — это определять, какой именно текст на странице описывает изображение. Тогда, если мы ищем по тексту, то мы сначала находим страницу, а потом, по положению текста и картинок, понимаем, какая картинка найденному тексту соответствует.[2] 

Если же ищем по картинке, то — наоборот, сначала ищем ближайшую картинку, а потом со страницы берём ближайший к ней текст. 

**Сейчас** гораздо чаще используются нейросетевые архитектуры, получающие общее векторное представление — и для картинок, и для текста[3] — в этом случае у нас есть возможность искать похожие объекты в этом векторном пространстве[4]. 

Иногда мы не хотим руками составлять поисковый запрос, но у нас есть какой-то документ, который нам понравился — например, статья по психологии. Так как я не специалист по психологии, я не могу составить грамотный поисковый запрос, чтобы поискать похожие статьи на эту же тему. Почему бы тогда не использовать прямо этот документ, как запрос? На помощь приходит "поиск похожих документов".[5] 

В этой области достаточно хорошо работают классические подходы с **TF-IDF и N-граммами**. Однако, как мы уже обсуждали, такие модели предполагают, что частоты словоупотреблений независимы, поэтому мы можем терять полноту из-за того, что не учитываем какие-то синонимы. 

Альтернативный подход — через **тематическое моделирование**.[6] Гигантская матрица размерности ["количество документов" на "количество слов"] факторизуется на две матрицы поменьше, и мы используем только первую из них для поиска документов. Этот механизм обладает большей полнотой, но уступает по точности первому подходу, так как при факторизации мы теряем информацию о специфических, достаточно редких словах.

[1] Frome, Andrea, et al. "Devise: A deep visual-semantic embedding model." Advances in neural information processing systems. 2013.  
[2] Feng, Yansong, and Mirella Lapata. "Automatic image annotation using auxiliary text information." Proceedings of ACL-08: HLT. 2008.  
[3] Wang, Liwei, Yin Li, and Svetlana Lazebnik. "Learning deep structure-preserving image-text embeddings." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.  
[4] Gomaa, Wael H., and Aly A. Fahmy. "A survey of text similarity approaches." International Journal of Computer Applications 68.13 (2013): 13-18.  
[5] Zubarev, Denis, and Ilya Sochenkov. "Using Sentence Similarity Measure for Plagiarism Source Retrieval." CLEF (Working Notes). 2014.  
[6] Ianina, Anastasia, Lev Golitsyn, and Konstantin Vorontsov. "Multi-objective topic modeling for exploratory search in tech news." Conference on Artificial Intelligence and Natural Language. Springer, Cham, 2017.  



Когда есть размеченная выборка или статистика посещений пользователя, есть возможность что-то обучить (например, градиентный бустинг или нейросеть). 

Сейчас постепенно набирают обороты попытки создать **более человеческий интерфейс** — например, человек может не задавать поисковый запрос как набор ключевых слов, вместо этого — писать текст на ему известном языке, задавать вопрос на естественном языке[1]. Например, "кто победил на прошлом чемпионате мира по футболу"? И система должна выдать непосредственно ответ на этот вопрос: например, "Франция". 

**Дано:**

- коллекция текстов, в которых мы будем искать ответы, 
- желательно иметь обучающую выборку, то есть набор пар "вопрос" и "текст, вместе с отметками — где в этом тексте находится ответ".[2] 

**Алгоритмы:**

Современные алгоритмы предназначены для поиска дословных ответов, то есть они не делают логический вывод. Если ответ встречается в тексте — они его выделяют, если нужно немного подумать — то лучше ничего не выделять. 

Мэйнстрим в этой области — нейросети.[4,5] Для них сейчас есть достаточно большие размеченные корпуса. Однако не стоит забывать и про классические методы[3] — глубокий лингвистический анализ и алгоритмы сопоставления графов. В ряде случаев они позволяют получить сопоставимое качество без обучающей выборки.

[1] Chen, Danqi, et al. "Reading Wikipedia to Answer Open-Domain Questions." Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). 2017.  
[2] Rajpurkar, Pranav, et al. "SQuAD: 100,000+ Questions for Machine Comprehension of Text." Proceedings of the 2016 Conference on Empirical Methods in Natural Language Processing. 2016.  
[3] Gliozzo, Alfio, et al. "Semantic technologies in IBM watson." Proceedings of the fourth workshop on teaching NLP and CL. 2013.  
[4] Shelmanov, A. O., et al. "Semantic-syntactic analysis for question answering and definition extraction." Scientific and Technical Information Processing 44.6 (2017): 412-423.  
[5] Ferrucci, David, et al. "Building Watson: An overview of the DeepQA project." AI magazine 31.3 (2010): 59-79.  

Список литературы:

- Chen, Danqi, et al. "Reading Wikipedia to Answer Open-Domain Questions." Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). 2017.
- Rajpurkar, Pranav, et al. "SQuAD: 100,000+ Questions for Machine Comprehension of Text." Proceedings of the 2016 Conference on Empirical Methods in Natural Language Processing. 2016.
- Ferrucci, David, et al. "Building Watson: An overview of the DeepQA project." AI magazine 31.3 (2010): 59-79.
- Gliozzo, Alfio, et al. "Semantic technologies in IBM watson." Proceedings of the fourth workshop on teaching NLP and CL. 2013.
- Shelmanov, A. O., et al. "Semantic-syntactic analysis for question answering and definition extraction." Scientific and Technical Information Processing 44.6 (2017): 412-423.

### <a id='toc2_1_3_'></a>[Извлечение структурированной информации](#toc0_)

Для того, чтобы компьютерная поддержка принятия решений работала и вообще имела смысл, необходимо составлять достаточно полные и актуальные базы фактов. Так уж сложилось, что в некоторых предметных областях значительная часть информации по-прежнему не публикуется в виде, пригодном для удобной обработки компьютерами, а публикуется, например, просто в текстах — например, новости. На помощь приходят методы извлечения структурированной информации. 

**Дано:**
- структура той информации, которую надо извлечь — набор полей и набор связей между ними. Это как схема базы данных. 
- набор текстов, в которых факты точно содержатся, но представлены в неструктурированном виде. 

Пример такой области — это новости, или медицинские карты, в которых врач простым текстом описывает, что происходит, какое лечение, и так далее.[1] 

**Алгоритмы:**
Методы извлечения информации заполняют поля значениями, найденными в тексте.[2] Нам здесь подходят **методы извлечения именованных сущностей** и связей между ними — о них мы немножко говорили в разделе про лингвистический анализ. В основе таких методов часто лежат системы правил, сопоставление со словарями, лингвистический анализ, а также **нейросети**, но только когда есть достаточно большие размеченные коллекции текстов — от 1000 примеров на каждую сущность.[3,4,5,6,7,8,9,10]

[1] Peng, Fuchun, and Andrew McCallum. "Information extraction from research papers using conditional random fields." Information processing & management 42.4 (2006): 963-979.  
[2] Surdeanu, Mihai, et al. "Stanford's distantly-supervised slot-filling system." (2011).  
[3] Kelly, Liadh, et al. "Overview of the share/clef ehealth evaluation lab 2014." International Conference of the Cross-Language Evaluation Forum for European Languages. Springer, Cham, 2014.  
[4] Mann, Gideon S., and David Yarowsky. "Multi-field information extraction and cross-document fusion." Proceedings of the 43rd annual meeting on association for computational linguistics. Association for Computational Linguistics, 2005.  
[5] Ratner, Alexander J., et al. "Snorkel: Fast training set generation for information extraction." Proceedings of the 2017 ACM International Conference on Management of Data. ACM, 2017.  
[6] Dernoncourt, Franck, Ji Young Lee, and Peter Szolovits. "NeuroNER: an easy-to-use program for named-entity recognition based on neural networks." Proceedings of the 2017 Conference on Empirical Methods in Natural Language Processing: System Demonstrations. 2017.  
[7] Dalvi, Bhavana, et al. "IKE-an interactive tool for knowledge extraction." Proceedings of the 5th Workshop on Automated Knowledge Base Construction. 2016.  
[8] Banko, Michele, et al. "Open information extraction from the web." Ijcai. Vol. 7. 2007  
[9] Huang, Zhiheng, Wei Xu, and Kai Yu. "Bidirectional LSTM-CRF models for sequence tagging." arXiv preprint arXiv:1508.01991 (2015).  
[10] Arkhipov, Mikhail Y., and Mikhail S. Burtsev. "Application of a hybrid Bi-LSTM-CRF Model to the task of Russian named entity recognition." Conference on Artificial Intelligence and Natural Language. Springer, Cham, 2017.  

### <a id='toc2_1_4_'></a>[Диалоговые системы](#toc0_)

Чат-бот — это не какой-то один алгоритм, это набор из различных алгоритмов, выполняющих разные функции, но работающих вместе. 

**Дано:**
- желаемый сценарий взаимодействия с пользователем (он может быть описан как явно, так и через примеры диалогов).[7,8] 
- цель общения - может быть как определена, так и быть свободной
- предметная область - может быть ограниченной, а может быть открытой. 

**Алгоритмы:**

Для построения чат-бота требуется сразу несколько алгоритмов для разных функций. Одна из таких функций — это определение интенции, то есть намерения пользователя: что он хочет от системы, с которой взаимодействует. Здесь подходят **методы классификации коротких текстов**. 

Также мы можем выдавать ответы посредством **поиска похожих текстов по базе**. 

**Нейросети** здесь также применяются — как для отдельных подзадач, так и для реализации чат-бота в рамках единого обучаемого алгоритма. Но это пока что на уровне эксперимента, скорее. 

### <a id='toc2_1_5_'></a>[Машинный перевод](#toc0_)

Другая задача, связанная с **генерацией текстов** — это машинный перевод. Чаще всего, перевод выполняется на уровне фраз или отдельных предложений. 

**Дано:**
- параллельные корпуса (тогда для каждого предложения на исходном языке известен хотя бы один вариант его перевода на целевом языке), 
- как вариант - непараллельные корпуса. Ц

**Алгоритмы:**

Центральное место в машинном переводе сейчас занимают **нейросети**.

Ранее использовались (а кое-где и сейчас используются) алгоритмы **статистического перевода на уровне отдельных фраз**.

[1] Chen, Hongshen, et al. "A survey on dialogue systems: Recent advances and new frontiers." Acm Sigkdd Explorations Newsletter 19.2 (2017): 25-35.  
[2] Serban, Iulian V., et al. "Building end-to-end dialogue systems using generative hierarchical neural network models." Thirtieth AAAI Conference on Artificial Intelligence. 2016.  
[3] Chuklin, Aleksandr, et al. "Proceedings of the 2018 EMNLP Workshop SCAI: The 2nd International Workshop on Search-Oriented Conversational AI." Proceedings of the 2018 EMNLP Workshop SCAI: The 2nd International Workshop on Search-Oriented Conversational AI. 2018.  
[4] Dinan, Emily, et al. "The second conversational intelligence challenge (convai2)." arXiv preprint arXiv:1902.00098 (2019).  
[5] Burtsev, Mikhail, et al. "DeepPavlov: open-source library for dialogue systems." Proceedings of ACL 2018, System Demonstrations. 2018.  
[6] Gupta, Arshit, John Hewitt, and Katrin Kirchhoff. "Simple, Fast, Accurate Intent Classification and Slot Labeling." arXiv preprint arXiv:1903.08268 (2019).  
[7] Serban, Iulian Vlad, et al. "A survey of available corpora for building data-driven dialogue systems: The journal version." Dialogue & Discourse 9.1 (2018): 1-49.  
[8] Lowe, Ryan Thomas, et al. "Training end-to-end dialogue systems with the ubuntu dialogue corpus." Dialogue & Discourse 8.1 (2017): 31-65.

### <a id='toc2_1_6_'></a>[Эксплоративный анализ](#toc0_)
**Дано:**
- коллекция текстов 

Нужно ответить на вопрос — "а что здесь вообще происходит?" То есть нужно понять, какие тематики есть, как они пересекаются и как меняются во времени. 

**Алгоритмы:**

Рабочая лошадка для таких задач — это **методы тематического моделирования**. Самые популярные методы — 
- латентное размещение Дирихле (LDA) 
- тематические модели с аддитивной регуляризацией (ARTM). 

Мы пробежались по основным прикладным задачам обработки текстов на естественном языке, а именно: классификации, поиску, извлечению информации, генерации текстов, эксплоративному анализу. На этом мы завершаем наш обзор задач обработки естественного языка.

[1] Вопросно-ответный поиск - описание соревнования и обзор решений, 2016 https://www.aclweb.org/anthology/S16-1083.pdf  
[2] Поиск похожих предложений, 2017 https://www.aclweb.org/anthology/S17-2028.pdf  
[3] Извлечение отношений, 2018 https://www.aclweb.org/anthology/S18-1130.pdf  
